In [1]:
n_dias:int = 3
import core
import trata_jogo_altair
import csv
from time import sleep,time
import os
import warnings
import pickle
import requests
from tqdm import tqdm
from datetime import datetime,timedelta,date,timezone
import numpy as np
warnings.filterwarnings('ignore')

ALTAIR_KEY = "195410-T5UVrbA62CyWYh"
carrossel = 0

keys = [ALTAIR_KEY]
forbidden_terms = ['esoccer','beach','amateur','indoor']
if not os.path.isdir(f"./upcoming_games"):
    os.makedirs(f"./upcoming_games")

print('Processando jogos do dia')
for d in range(n_dias+1):
    formatted_date = (datetime.now(timezone.utc) + timedelta(days=d)).strftime("%Y%m%d") 
    page = 1
    max_pages = float('inf')
    while page <= max_pages:
        url = f"https://api.b365api.com/v3/events/upcoming?sport_id=1&token={keys[carrossel]}&day={formatted_date}&skip_esports=yes&page={page}"

        response = requests.get(url).json()

        if page == 1:
            max_pages = response['pager']['total'] // response['pager']['per_page']
            
            if (response['pager']['total'] % response['pager']['per_page']) >0:
                max_pages += 1
            
            pbar = tqdm(total=max_pages,desc=(datetime.now(timezone.utc) + timedelta(days=d)).strftime("%d/%m/%Y") )


        for x in response['results']:
            proced = 1

            for ft in forbidden_terms:
                if ft in x['league']['name'].lower():
                    proced = 0
            if x['time_status'] != "3" and proced == 1:
                if not f"{x['id']}.pkl" in os.listdir(f"./upcoming_games"):
                    start = time()
                    jogo = core.trata_jogo(x,keys[carrossel])
                    with open(f"./upcoming_games/{x['id']}.pkl", "wb") as arquivo:
                        pickle.dump(jogo, arquivo)
                    end = time() - start 
                    if end < (1/len(keys)):
                        sleep((1/len(keys)) - end)
                    
                    carrossel += 1
                    if carrossel >= len(keys):
                        carrossel = 0
        
        page += 1
        pbar.update(1)

    pbar.close()

#odds fechamento
url = f"https://api.b365api.com/v3/events/inplay?sport_id=1&token={keys[carrossel]}&skip_esports=yes"

response = requests.get(url).json()

for x in response['results']:
    proced = 1

    for ft in forbidden_terms:
        if ft in x['league']['name'].lower():
            proced = 0
    if x['time_status'] != "3" and proced == 1:
        if not f"{x['id']}.pkl" in os.listdir(f"./upcoming_games"):
            start = time()
            jogo = core.trata_jogo(x,keys[carrossel])
            with open(f"./upcoming_games/{x['id']}.pkl", "wb") as arquivo:
                pickle.dump(jogo, arquivo)
            end = time() - start 
            if end < (1/len(keys)):
                sleep((1/len(keys)) - end)
            
            carrossel += 1
            if carrossel >= len(keys):
                carrossel = 0

#Truncando jogos do dia
with open("./upcoming_games.csv", mode='w', newline='', encoding='utf-8') as arquivo_csv:
    pass

with open("./upcoming_games.csv", mode='a', newline='', encoding='utf-8') as arquivo_csv:
    escritor_csv = csv.writer(arquivo_csv)
    escritor_csv.writerow(trata_jogo_altair.colunas_dia)
    for arquivo in os.listdir('./upcoming_games'):
        with open('./upcoming_games/'+arquivo, "rb") as fp:
            jogo = pickle.load(fp)
            jogo = trata_jogo_altair.trata_jogo(jogo)
        linha = []
        for coluna in trata_jogo_altair.colunas_dia:
            linha.append(jogo.get(coluna, None))
        escritor_csv.writerow(linha)

        os.remove(fp.name)
# import pandas as pd
# pd.read_csv("./upcoming_games.csv").sort_values("utc_time").to_excel("jogos_do_dia.xlsx",index=False)
print("Pronto!")

Processando jogos do dia


16/08/2024: 100%|██████████| 7/7 [05:25<00:00, 46.55s/it]


Pronto!
